# Market Price Data Collection

This notebook scrapes and combines weekly price data from Mongolia's National Statistics Office. Each week they record average prices for various staple products around Ulaanbaatar. 

Link to web page: https://1212.mn/BookLibrary.aspx?category=004

This notebook will be run on a schedule to allow for the dataset to be updated monthly.

In [ ]:
import pandas as pd
import glob
import requests
from bs4 import BeautifulSoup
import shutil
import os
import datetime

### Scraping data
* Link to datasets: https://1212.mn/BookLibrary.aspx?category=004

In [ ]:
def download_file(url, name):
    with requests.get(url, stream=True) as r:
        with open(name, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

    return name

In [ ]:
if os.path.isdir('raw'):
    shutil.rmtree('raw', ignore_errors=True)
    os.mkdir('raw')

In [ ]:
URL = "https://1212.mn/BookLibrary.aspx?category=004"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")


In [ ]:
# table = soup.findAll('tr')
table = soup.find('tbody')

In [ ]:
tbl_rows = table.findAll('tr')
for row in tbl_rows:
    if row.find('img')['src'] == '/img/icon/xls_icon.gif': #get only xlsx files
        link_to_xlsx = row.a['href']
        date = row.find('img')['alt']
        if date > '2019-01-01': #date from 2012-01-01
            download_file(link_to_xlsx, 'raw/{}.xlsx'.format(date))
        else:
            break

### Merging datasets

In [ ]:
def sheet_df(sheet):
    sheet = pd.read_excel(sheet, sheet_name=2, skiprows=4, index_col=[0, 1])
    sheet = sheet.dropna(axis=0, how='all')
    sheet = sheet.dropna(axis=1, how='all')
    sheet = sheet.reset_index()
    
    sheet['Барааны нэр'] = sheet['Барааны нэр'].str.strip()
    sheet['Барааны нэр'] = sheet['Барааны нэр'] + ' ' + sheet.iloc[:,2].fillna(value='')
    sheet['Барааны нэр'] = sheet['Барааны нэр'].str.strip()
    sheet = sheet.drop(columns=['№', 'Unnamed: 2','Дундаж үнэ'])
    return sheet

In [ ]:
path =r'raw/'
filenames = glob.glob(path + "*.xlsx")

In [ ]:
main_df = {}
for filename in filenames:
    try:
        with pd.ExcelFile(filename) as reader:
            date = filename[4:14] #get only date from file name
            main_df[date] = sheet_df(reader)
    except:
        print(filename)

In [ ]:
main_df = pd.concat(main_df, axis=0)
main_df = main_df.reset_index()
main_df = main_df.drop(['level_1'], axis=1)
main_df = main_df.rename({'level_0': 'date'}, axis=1)
main_df = pd.melt(main_df, id_vars=['date', 'Барааны нэр', 'хэмжих нэгж'], value_vars=main_df.columns.drop(['date', 'Барааны нэр', 'хэмжих нэгж']))
main_df.head()

,date,Барааны нэр,хэмжих нэгж,variable,value
0,2020-08-05,"""Алтан тариа"" гурил дээд",кг,"""Баянзүрх"" зах",1400.0
1,2020-08-05,"""Алтан тариа"" гурил 1-р зэрэг",кг,"""Баянзүрх"" зах",1200.0
2,2020-08-05,"""Алтан тариа"" савласан гурил дээдийн дээд",кг,"""Баянзүрх"" зах",2200.0
3,2020-08-05,"""Алтан тариа"" савласан гурил дээд",кг,"""Баянзүрх"" зах",1400.0
4,2020-08-05,"""Алтан тариа"" савласан гурил 1-р зэрэг",кг,"""Баянзүрх"" зах",1200.0


In [ ]:
main_df['date'] = pd.to_datetime(main_df['date'])

In [ ]:
main_df = main_df.sort_values(by='date').reset_index(drop=True)

In [ ]:
print(main_df['date'].min())
print(main_df['date'].max())

2019-01-02 00:00:00
2021-12-08 00:00:00


## Translation and Merging of Markets

We will have two versions of this dataset, English and Mongolian. Also, some markets have been merged or moved (Mercury moved to Ikh Nayad, Max was renamed to M Mart).

In [ ]:
main_df.head()

,date,Барааны нэр,хэмжих нэгж,variable,value
0,2019-01-02,"Байцаа, монгол",кг,Имарт,790.0
1,2019-01-02,"""Алтан тариа"" савласан гурил 1-р зэрэг",кг,"""Барс"" хт",1000.0
2,2019-01-02,"""Алтан тариа"" савласан гурил 2-р зэрэг",кг,"""Барс"" хт",800.0
3,2019-01-02,"""Алейка"" дээд гурил",кг,"""Барс"" хт",NaN
4,2019-01-02,Сүү Сүү ХК 1л,литр,"""Барс"" хт",2700.0


Split off version for Mongolian and English

In [ ]:
mgl_df = main_df.rename(columns={'variable':'market', 'value':'price',
                                 'Барааны нэр':'product','хэмжих нэгж':'size'})

In [ ]:
eng_df = main_df.rename(columns={'variable':'market', 'value':'price',
                                 'Барааны нэр':'product','хэмжих нэгж':'size'})

### Mongolian Dataframe Edits

In [ ]:
market_translate_mn = {'"Меркури" зах':'"Меркури" зах/Их наяд ХТ', 'Их наяд ХТ':'"Меркури" зах/Их наяд ХТ',
                       'Макс Супер-маркет':'Макс/М Март Супер-маркет'}

In [ ]:
mgl_df['market'] = mgl_df['market'].replace(market_translate_mn)

The Suu brand product names are a bit wonky because of how they were merged. Let's fix that.

In [ ]:
mgl_df['product'].unique()

array(['Байцаа, монгол', '"Алтан тариа" савласан гурил 1-р зэрэг',
       '"Алтан тариа" савласан гурил 2-р зэрэг', '"Алейка" дээд гурил',
       'Сүү Сүү ХК 1л', 'Сүү Сүү ХК 0.5л', 'Сүү Задгай',
       'Тараг, Цөцгийтэй', '"Алтан тариа" савласан гурил дээд',
       'Талх Атар', 'Талх Хар талх', 'Будаа цагаан', 'Будаа шар',
       'Цөцгийн тос, "Сүү" ХК', 'Ургамлын тос', 'Ногоон цай, Гүрж',
       'Элсэн чихэр', 'Талх Зууван чех',
       '"Алтан тариа" савласан гурил дээдийн дээд',
       '"Алтан тариа" гурил 1-р зэрэг', '"Алтан тариа" гурил дээд',
       'Хонины мах ястай', 'Хонины мах цул', 'Үхрийн мах ястай',
       'Үхрийн мах цул', 'Өндөг, ОХУ', 'Адууны мах, ястай',
       'Ямааны мах, ястай', 'Төмс, монгол', 'Лууван, монгол',
       'Хүрэн манжин, монгол', 'Сонгино, хятад', 'Алим, фүжи',
       'Нөөцийн мах, ястай хонь', 'Нөөцийн мах, ястай үхэр',
       'Нөөцийн мах, ястай ямаа'], dtype=object)

In [ ]:
mgl_df['product'] = mgl_df['product'].str.replace('Сүү Сүү', 'Сүү')

Since we renamed two markets to have the same name (Mercury and Ikh Nayad), we have duplicate dates and products for the same markets, but with NaNs. This is because Mercury closed and the shops moved to Ikh Nayad. Let's merge those.

In [ ]:
mgl_df = mgl_df.groupby(by=['date','product','market']).first().reset_index()

In [ ]:
mgl_df

,date,product,market,size,price
0,2019-01-02,"""Алейка"" дээд гурил","""Барс"" хт",кг,NaN
1,2019-01-02,"""Алейка"" дээд гурил","""Баянзүрх"" зах",кг,NaN
2,2019-01-02,"""Алейка"" дээд гурил","""Бөмбөгөр"" хт",кг,NaN
3,2019-01-02,"""Алейка"" дээд гурил","""Меркури"" зах/Их наяд ХТ",кг,NaN
4,2019-01-02,"""Алейка"" дээд гурил","""Сансар"" Супер-маркет",кг,NaN
...,...,...,...,...,...
61927,2021-12-08,"Өндөг, ОХУ","""Хүчит шонхор"" зах",ширхэг,400.0
61928,2021-12-08,"Өндөг, ОХУ",Имарт,ширхэг,NaN
61929,2021-12-08,"Өндөг, ОХУ",Макс/М Март Супер-маркет,ширхэг,500.0
61930,2021-12-08,"Өндөг, ОХУ",Номин Супер-маркет,ширхэг,605.0


The Mongolian dataset is now done, and can be written to a CSV file.

In [ ]:
mgl_df.to_csv('data/prices_mn.csv', index=False)

### English Dataframe Edits

In [ ]:
market_translate_en = {'"Барс" хт':'Bars', '"Баянзүрх" зах':'Bayanzurkh', '"Бөмбөгөр" хт':'Bumbugur', '"Меркури" зах':'Mercury/Ikh Nayad',
                       '"Сансар" Супер-маркет':'Sansar', '"Саруул" зах':'Saruul', '"Таван эрдэнэ"   зах':'Tavan Erdene',
                       '"Хар хорин" зах':'Kharkhorin', '"Хүчит шонхор"  зах':'Khuchit Shonkhor', 'Имарт':'Emart', 'Их наяд ХТ':'Mercury/Ikh Nayad',
                       'Макс Супер-маркет':'Max/M Mart', 'Номин Супер-маркет':'Nomin', 'Оргил Супер-маркет':'Orgil'}

In [ ]:
eng_df['market'] = eng_df['market'].replace(market_translate_en)

In [ ]:
eng_df['size'].unique()

array(['кг', 'литр', 'ширхэг', '300 гр', '1 литр'], dtype=object)

In [ ]:
size_translate = {'ширхэг':'each', 'кг':'kg', '1 литр':'1 liter', '300 гр':'300 g', 'литр':'liter'}

In [ ]:
eng_df['size'] = eng_df['size'].replace(size_translate)

In [ ]:
eng_df['product'].unique()

array(['Байцаа, монгол', '"Алтан тариа" савласан гурил 1-р зэрэг',
       '"Алтан тариа" савласан гурил 2-р зэрэг', '"Алейка" дээд гурил',
       'Сүү Сүү ХК 1л', 'Сүү Сүү ХК 0.5л', 'Сүү Задгай',
       'Тараг, Цөцгийтэй', '"Алтан тариа" савласан гурил дээд',
       'Талх Атар', 'Талх Хар талх', 'Будаа цагаан', 'Будаа шар',
       'Цөцгийн тос, "Сүү" ХК', 'Ургамлын тос', 'Ногоон цай, Гүрж',
       'Элсэн чихэр', 'Талх Зууван чех',
       '"Алтан тариа" савласан гурил дээдийн дээд',
       '"Алтан тариа" гурил 1-р зэрэг', '"Алтан тариа" гурил дээд',
       'Хонины мах ястай', 'Хонины мах цул', 'Үхрийн мах ястай',
       'Үхрийн мах цул', 'Өндөг, ОХУ', 'Адууны мах, ястай',
       'Ямааны мах, ястай', 'Төмс, монгол', 'Лууван, монгол',
       'Хүрэн манжин, монгол', 'Сонгино, хятад', 'Алим, фүжи',
       'Нөөцийн мах, ястай хонь', 'Нөөцийн мах, ястай үхэр',
       'Нөөцийн мах, ястай ямаа'], dtype=object)

In [ ]:
product_translate = {'Өндөг, ОХУ':'Egg, Russian', 'Будаа шар':'Yellow Rice', 'Цөцгийн тос, "Сүү" ХК':'Butter, Suu LTD', 'Ургамлын тос':'Vegatable Oil',
                     'Ногоон цай, Гүрж':'Gree Tea, Georgia', 'Элсэн чихэр':'Sugar', 'Хонины мах ястай':'Mutton w/Bones',
                     'Хонины мах цул':'Mutton Boneless', 'Үхрийн мах ястай':'Beef w/Bones', 'Үхрийн мах цул':'Beef Boneless',
                     'Адууны мах, ястай':'Horse Meat w/Bones', 'Ямааны мах, ястай':'Goat Meat w/Bones', 'Будаа цагаан':'White Rice',
                     'Төмс, монгол':'Potato, Mongolian', 'Байцаа, монгол':'Cabbage, Mongolian', 'Хүрэн манжин, монгол':'Beet, Mongolian',
                     'Сонгино, хятад':'Onion, China', 'Алим, фүжи':'Apple, Fuji', '"Алтан тариа" гурил дээд':'Altan Taria Flour, High Grade',
                     '"Алтан тариа" гурил 1-р зэрэг':'Altan Taria Flour, First Grade',
                     '"Алтан тариа" савласан гурил дээдийн дээд':'Altan Taria Packaged Flour, Top Grade',
                     '"Алтан тариа" савласан гурил дээд':'Altan Taria Packaged Flour, High Grade',
                     '"Алтан тариа" савласан гурил 1-р зэрэг':'Altan Taria Packaged Flour, 1st Grade',
                     '"Алтан тариа" савласан гурил 2-р зэрэг':'Altan Taria Packaged Flour, 2nd Grade', 
                     'Лууван, монгол':'Carrot, Mongolian', 'Талх Хар талх':'Brown Bread', 'Талх Зууван чех':'Round Bread, Czech Style', 
                     'Талх Атар':'Atar Brand Bread', '"Алейка" дээд гурил':'Aleika High Grade Flour', 'Сүү Сүү ХК 1л':'Suu Brand Milk, 1l', 
                     'Сүү Сүү ХК 0.5л':'Suu Brand Milk, 0.5l', 'Сүү Задгай':'Milk, Bulk', 'Тараг, Цөцгийтэй':'Yogurt', 
                     'Нөөцийн мах, ястай үхэр':'Reserve meat, beef w/bones',
                     'Нөөцийн мах, ястай хонь':'Reserve meat, mutton w/bones', 
                     'Нөөцийн мах, ястай ямаа':'Reserve meat, goat w/bones'}

In [ ]:
eng_df['product'] = eng_df['product'].map(product_translate)

Since we renamed two markets to have the same name (Mercury and Ikh Nayad), we have duplicate dates and products for the same markets, but with NaNs. This is because Mercury closed and the shops moved to Ikh Nayad. Let's merge those.

In [ ]:
eng_df = eng_df.groupby(by=['date','product','market']).first().reset_index()

In [ ]:
eng_df.to_csv('data/prices_en.csv', index=False)

## Upload to Kaggle

We only do this once, so the code is commented out. But keeping here for fun.

In [ ]:
#!kaggle datasets init -p data

Now edit the JSON file with what you want.

In [ ]:
#!kaggle datasets create -p data

## Update Dataset

We will only do the creation once, so now we can comment out the two lines above. When updated the dataset we will down the metadata file from Kaggle.com since we will edit the metadata on Kaggle.

In [ ]:
!kaggle datasets metadata -p data robertritz/ub-market-prices

Downloaded metadata to data/dataset-metadata.json


In [ ]:
date = datetime.datetime.today()

In [ ]:
!kaggle datasets version -p data -m "Updated dataset. {date}"

Starting upload for file prices_mn.csv
100%|██████████████████████████████████████| 5.32M/5.32M [00:01<00:00, 3.41MB/s]
Upload successful: prices_mn.csv (5MB)
Starting upload for file prices_en.csv
100%|██████████████████████████████████████| 2.97M/2.97M [00:01<00:00, 2.66MB/s]
Upload successful: prices_en.csv (3MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/robertritz/ub-market-prices


Now the updated dataset should be live!

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c29b651-5799-4ecf-8ca8-8e3f274961e2' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>